# ENEM 2018

# Business Understanding

The ENEM is the largest educational exam in Brazil, and its microdata base contains millions of records that represent a snapshot of Brazilian education. However:

❗ **Problem (Pain Point)**

Today, schools, education networks, and analysts have great difficulty in anticipating student performance before the exam is administered. This leads to:

- Reactive pedagogical planning (corrections after the results, not before).
- Difficulty in customizing study paths.
- Impossibility of simulating scenarios (e.g., impact of attendance, type of school, socioeconomic profiles).
- Low predictability of school performance, harming public policies.

💡 **Opportunity**

Build a predictive model capable of estimating the four objective scores of the ENEM (excluding the essay) based on:

- student profile,
- socioeconomic conditions,
- school information,
- exam status (attendance, language, etc.).

With this, schools and administrators can:

- Create personalized pedagogical plans;
- Anticipate at-risk groups with low performance;
- Simulate the impact of educational policies;
- Increase resource alignment.

✅ **Solution**

Create a model that receives a single set of features and generates four predicted scores simultaneously:

- NU_NOTA_CN
- NU_NOTA_CH
- NU_NOTA_LC
- NU_NOTA_MT

**Candidate models:**
- Lasso Multioutput
- Random Forest Multioutput
- XGBoost Multioutput

🟩 **Advantages**

- Integration between the four competencies (the scores are correlated).
- Unique and reduced architecture.
- More consistent and easier to explain model.

# Data Understanding

## Overview

In [ ]:
import os
import pandas as pd
import numpy as np
from scipy.stats import kstest, shapiro, norm
from typing import List, Tuple, Dict, Any
from matplotlib.figure import Figure
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import probplot
import sqlite3
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.multioutput import MultiOutputRegressor
from sklearn.linear_model import Lasso
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import NearestNeighbors
from xgboost import XGBRegressor
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingGridSearchCV
from sklearn.model_selection import KFold
from sklearn.metrics.pairwise import cosine_similarity
import shap
import nest_asyncio
import asyncio
from pyngrok import ngrok
import uvicorn
from uvicorn.config import Config
from uvicorn.server import Server
from fastapi import FastAPI, Depends, HTTPException, status
from fastapi.security import HTTPBearer, HTTPAuthorizationCredentials
from pydantic import BaseModel
import pickle
import nest_asyncio
import threading
import requests
import json
import warnings
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.data import HeteroData
from torch.optim.lr_scheduler import CosineAnnealingLR
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts
from torch_geometric.nn import HeteroConv, SAGEConv
from torch_geometric.loader import NeighborLoader
from collections import defaultdict


warnings.filterwarnings("ignore", category=FutureWarning, module="seaborn")
pd.set_option('display.max_columns', None)

In [ ]:
conn = sqlite3.connect('database.db')

In [ ]:
# Filtering only people who have taken all four tests and who are not practice test takers.
query = """
SELECT *
FROM enem
WHERE
(TP_PRESENCA_CN = 1)
AND (TP_PRESENCA_CH = 1)
AND (TP_PRESENCA_LC = 1)
AND (TP_PRESENCA_MT = 1)
AND (NU_NOTA_CN > 0 and NU_NOTA_CN is not null)
AND (NU_NOTA_CH > 0 and NU_NOTA_CH is not null)
AND (NU_NOTA_LC > 0 and NU_NOTA_LC is not null)
AND (NU_NOTA_MT > 0 and NU_NOTA_MT is not null)
AND (IN_TREINEIRO == 0)
"""

df_raw = pd.read_sql_query(query, conn)
df_raw

In [ ]:
def multi_stratified_sample(
    df: pd.DataFrame,
    strata_cols: List[str],
    n_samples: int,
    random_state: int = 42
) -> pd.DataFrame:
    """
    Performs stratified sampling in multiple columns.

    Parameters:
    -----------
    df : pd.DataFrame
        Input DataFrame with millions of rows.
    strata_cols : list
        List of columns which defines the strata.
    n_samples : int
        Number of desired samples.
    random_state : int
        Seed for reproducibility.

    Return:
    --------
    pd.DataFrame : stratified sample.
    """
    # Ensure reproducibility
    np.random.seed(random_state)

    # Simple Verifcation
    if n_samples > len(df):
        raise ValueError("n_samples cannot be greater than the size of the DataFrame.")

    # Create groups by strata
    grouped = df.groupby(strata_cols)

    # Size of each group (stratum)
    group_sizes = grouped.size()

    # Proportion of each stratum
    proportions = group_sizes / group_sizes.sum()

    # Number of samples per stratum (proportional)
    samples_per_group = (proportions * n_samples).round().astype(int)

    # Ensure that the final sum is exactly n_samples
    diff = n_samples - samples_per_group.sum()

    if diff != 0:
        # Adjust the largest strata until they exactly match n_samples
        largest_groups = samples_per_group.sort_values(ascending=False)
        samples_per_group.loc[largest_groups.index[:abs(diff)]] += np.sign(diff)

    # Perform stratified sampling
    sampled_df = (
        grouped
        .apply(lambda x: x.sample(n=samples_per_group.loc[x.name], random_state=random_state, replace=False))
        .reset_index(drop=True)
    )

    return sampled_df

In [ ]:
df = df_raw.copy()

df = multi_stratified_sample(df, ['SG_UF_ESC','TP_ST_CONCLUSAO','TP_ESCOLA','TP_ENSINO','TP_COR_RACA','TP_SEXO'], 30000, random_state=42)

relevant_columns = ['TP_FAIXA_ETARIA', 'TP_SEXO','TP_ESTADO_CIVIL', 'TP_COR_RACA','TP_ST_CONCLUSAO',
                    'TP_ESCOLA', 'TP_ENSINO','SG_UF_ESC','TP_LOCALIZACAO_ESC', 'NU_NOTA_CN',
                    'NU_NOTA_CH', 'NU_NOTA_LC', 'NU_NOTA_MT', 'NU_NOTA_REDACAO', 'Q001', 'Q002', 'Q003', 'Q004',
                    'Q005', 'Q006', 'Q007', 'Q008', 'Q009', 'Q010', 'Q011', 'Q012', 'Q013',
                    'Q014', 'Q015', 'Q016', 'Q017', 'Q018', 'Q019', 'Q020', 'Q021', 'Q022',
                    'Q023', 'Q024', 'Q025','Q027']

df = df[relevant_columns]

df.head(5)

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
df.describe().T

## Checking for missing data

In [ ]:
print("Missing values in the dataset:",df.isnull().sum().sum())

## Checking for duplicates

In [ ]:
# Check if there are duplicates
duplicates = df.duplicated()

if duplicates.any():
    # Option 1: Print the entire row with duplicates
    print(df[duplicates])

    # Option 2: Count the number of duplicates
    print(duplicates.sum(), "duplicate rows found")
else:
    print("No duplicate rows found in the dataset")

In [ ]:
def detect_outliers_multi_boxplot(
    df: pd.DataFrame,
    columns: List[str],
    figsize: Tuple[int, int] = (16, 6),
    title_size: int = 18
) -> Figure:
    """
    Identifies outliers for multiple numeric columns using the IQR method
    and plots side-by-side boxplots with a polished executive-style layout.

    Parameters:
    -----------
    df : pd.DataFrame
        Input dataset
    columns : list
        List of numeric columns to analyze
    figsize : tuple
        Size of the figure
    title_size : int
        Title font size

    Returns:
    --------
    pd.DataFrame
        Summary table with quartiles, bounds, outlier count and percentage
    """

    stats_list = []

    sns.set(style="whitegrid")
    plt.figure(figsize=figsize)

    # Colors
    box_color = "#4C72B0"
    outlier_color = "#DD8452"

    # Create subplots
    num_cols = len(columns)
    fig, axes = plt.subplots(1, num_cols, figsize=figsize)

    # Ensure axes is iterable even if 1 column
    if num_cols == 1:
        axes = [axes]

    for i, col in enumerate(columns):

        series = df[col].dropna()

        # Quartiles & IQR
        Q1 = series.quantile(0.25)
        Q3 = series.quantile(0.75)
        IQR = Q3 - Q1

        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR

        outliers = series[(series < lower_bound) | (series > upper_bound)]

        # Store stats
        stats_list.append({
            "column": col,
            "Q1": Q1,
            "Q3": Q3,
            "IQR": IQR,
            "lower_bound": lower_bound,
            "upper_bound": upper_bound,
            "outlier_count": len(outliers),
            "outlier_percentage": round(100 * len(outliers) / len(series), 2)
        })

        # Plot boxplot
        sns.boxplot(
            y=series,
            ax=axes[i],
            color=box_color,
            width=0.4,
            fliersize=7,
            flierprops=dict(
                marker='o',
                markerfacecolor=outlier_color,
                markersize=8,
                linestyle='none'
            )
        )

        axes[i].set_title(f"{col}", fontsize=title_size, fontweight="bold")
        axes[i].grid(axis='y', linestyle="--", alpha=0.4)

        # Add lines for bounds
        axes[i].axhline(lower_bound, color="#C44E52", linestyle="--", linewidth=1.5)
        axes[i].axhline(upper_bound, color="#55A868", linestyle="--", linewidth=1.5)

    plt.suptitle("Outlier Analysis", fontsize=title_size+2, fontweight="bold")
    plt.tight_layout(rect=[0, 0, 1, 0.95])
    plt.show()

    # Convert statistics to DataFrame
    summary_df = pd.DataFrame(stats_list)

    # Print summary
    print("\n===== Outlier Analysis Summary =====")
    print(summary_df.to_string(index=False))

    return summary_df

In [ ]:
detect_outliers_multi_boxplot(df, ['NU_NOTA_CN','NU_NOTA_CH','NU_NOTA_LC','NU_NOTA_MT'], figsize=(10, 5), title_size=18)

In [ ]:
# Creating a copy of original dataset to change name variables for readability
df_EDA = df.copy()

# Creating 6 subplots
fig, ax = plt.subplots(2, 3, figsize=(19, 18))

# Age range
df_EDA = df_EDA.sort_values(by='TP_FAIXA_ETARIA')
df_EDA['TP_FAIXA_ETARIA'] = df_EDA['TP_FAIXA_ETARIA'].map({
    1:'<17',
    2:'17',
    3:'18',
    4:'19',
    5:'20',
    6:'21',
    7:'22',
    8:'23',
    9:'24',
    10:'25',
    11:'26-30',
    12:'31-35',
    13:'36-40',
    14:'41-45',
    15:'46-50',
    16:'51-55',
    17:'56-60',
    18:'61-65'
})

ax1 = ax[0,0]
sns.countplot(x=df_EDA['TP_FAIXA_ETARIA'], ax=ax1)

ax1.set(xlabel='Age range (years)',
        ylabel='Number of candidates',
        title='TP_FAIXA_ETARIA: Age range')

for label in ax1.get_xticklabels():
    label.set_rotation(60)
ax1.grid(False)

# Sex
df_EDA['TP_SEXO'] = df_EDA['TP_SEXO'].map({'F':'Female','M':'Male'})
ax2 = ax[0,1]
sns.countplot(x=df_EDA['TP_SEXO'], ax=ax2)

ax2.set(xlabel='Sex', ylabel='Number of candidates',title='TP_SEXO: Sex')
ax2.grid(False)

# Ethnicity
df_EDA['TP_COR_RACA'] = df_EDA['TP_COR_RACA'].map({
    0:'Not declared',
    1:'White',
    2:'Black',
    3:'Pardo',
    4:'Yellow',
    5:'Indiginous'
})
ax3 = ax[0,2]
sns.countplot(x=df_EDA['TP_COR_RACA'], ax=ax3, order=df_EDA['TP_COR_RACA'].value_counts(ascending=False).index)

ax3.set(xlabel='Ethnicity', ylabel='Number of candidates',title='TP_COR_RACA: Ethnicity')
for label in ax3.get_xticklabels():
    label.set_rotation(20)
ax3.grid(False)


# School (Private, Public)
df_EDA['TP_ESCOLA'] = df_EDA['TP_ESCOLA'].map({1:'Not declared',2:'Public',3:'Exterior',4:'Private'})
ax4 = ax[1,0]
sns.countplot(x=df_EDA['TP_ESCOLA'], ax=ax4, order=df_EDA['TP_ESCOLA'].value_counts(ascending=False).index)

ax4.set(xlabel='Type of school', ylabel='Number of candidates',title='TP_ESCOLA: Type of School')
for label in ax4.get_xticklabels():
    label.set_rotation(20)
ax4.grid(False)

# Situation high school (finished, finishing)
df_EDA['TP_ST_CONCLUSAO'] = df_EDA['TP_ST_CONCLUSAO'].map({
    1:'Yes',
    2:'No, will in 2018',
    3:'No, will after 2018',
    4:'No and will not'
    })
ax5 = ax[1,1]
sns.countplot(x=df_EDA['TP_ST_CONCLUSAO'], ax=ax5, order=df_EDA['TP_ST_CONCLUSAO'].value_counts(ascending=False).index)

ax5.set(xlabel='Finished high school?', ylabel='Number of candidates',title='TP_ST_CONCLUSAO: High School Situation')
for label in ax5.get_xticklabels():
    label.set_rotation(20)
ax5.grid(False)

# Family income
df_EDA = df_EDA.sort_values(by='Q006')
ax6 = ax[1,2]
sns.countplot(x=df_EDA['Q006'], ax=ax6)
ax6.set(xlabel='Family income class (A: 0 to Q: > R$19K)', ylabel='Number of candidates',title='Q006: Family Income')
for label in ax6.get_xticklabels():
    label.set_rotation(90)
ax6.grid(False)

plt.show()

In [ ]:
# Creating subplots
fig, ax = plt.subplots(1, 3, figsize=(18, 6))

# Natural Sciences
ax1 = ax[0]
sns.histplot(x=df_EDA['NU_NOTA_CN'], bins=int(180/5), color='blue', edgecolor='black', ax=ax1, alpha=0.2)
mean_cn = df_EDA['NU_NOTA_CN'].mean()
median_cn = df_EDA['NU_NOTA_CN'].median()
ax1.axvline(x=mean_cn, color='red', linestyle='--', label='Mean')
ax1.axvline(x=median_cn, color='green', linestyle='-', label='Median')

ax1.set(xlabel='Grades', ylabel='Number of candidates', title='NU_NOTA_CN: Natural Sciences grade')
for label in ax1.get_xticklabels():
    label.set_rotation(20)
ax1.legend()
ax1.grid(False)

# Human Sciences
ax2 = ax[1]
sns.histplot(x=df_EDA['NU_NOTA_CH'], bins=int(180/5), color='yellow', edgecolor='black', ax=ax2, alpha=0.2)
mean_cn = df_EDA['NU_NOTA_CH'].mean()
median_cn = df_EDA['NU_NOTA_CH'].median()
ax2.axvline(x=mean_cn, color='red', linestyle='--', label='Mean')
ax2.axvline(x=median_cn, color='green', linestyle='-', label='Median')

ax2.set(xlabel='Grades', ylabel='Number of candidates', title='NU_NOTA_CH: Human Sciences grade')
for label in ax2.get_xticklabels():
    label.set_rotation(20)
ax2.legend()
ax2.grid(False)

# Math
ax3 = ax[2]
sns.histplot(x=df_EDA['NU_NOTA_MT'], bins=int(180/5), color='pink', edgecolor='black', ax=ax3, alpha=0.2)
mean_cn = df_EDA['NU_NOTA_MT'].mean()
median_cn = df_EDA['NU_NOTA_MT'].median()
ax3.axvline(x=mean_cn, color='red', linestyle='--', label='Mean')
ax3.axvline(x=median_cn, color='green', linestyle='-', label='Median')
ax3.set(xlabel='Grades', ylabel='Number of candidates', title='NU_NOTA_MT: Mathematics grade')
for label in ax3.get_xticklabels():
    label.set_rotation(20)
ax3.legend()
ax3.grid(False)

plt.show()


fig, ax = plt.subplots(1, 2, figsize=(15, 6))
# Languages
ax4 = ax[0]
sns.histplot(x=df_EDA['NU_NOTA_LC'], bins=int(180/5), color='green', edgecolor='black', ax=ax4, alpha=0.2)
mean_cn = df_EDA['NU_NOTA_LC'].mean()
median_cn = df_EDA['NU_NOTA_LC'].median()
ax4.axvline(x=mean_cn, color='red', linestyle='--', label='Mean')
ax4.axvline(x=median_cn, color='green', linestyle='-', label='Median')
ax4.set(xlabel='Grades', ylabel='Number of candidates', title='NU_NOTA_LC: Language Abilities grade')
for label in ax4.get_xticklabels():
    label.set_rotation(20)
ax4.legend()
ax4.grid(False)


# Natural Sciences
ax5 = ax[1]
sns.histplot(x=df_EDA['NU_NOTA_REDACAO'], bins=int(180/5), color='purple', edgecolor='black', ax=ax5, alpha=0.2)
mean_cn = df_EDA['NU_NOTA_REDACAO'].mean()
median_cn = df_EDA['NU_NOTA_REDACAO'].median()
ax5.axvline(x=mean_cn, color='red', linestyle='--', label='Mean')
ax5.axvline(x=median_cn, color='green', linestyle='-', label='Median')

ax5.set(xlabel='Grades', ylabel='Number of candidates', title='NU_NOTA_REDACAO: Critical essay grade')
for label in ax5.get_xticklabels():
    label.set_rotation(20)
ax5.legend()
ax5.grid(False)


plt.show()

<div class="alert alert-info">
<strong><div style="color: rgb(0, 0, 0);">📌  Insights:</div></strong> <br>
<div style="color: rgb(0, 0, 0);">→ The highest mean was for Human Sciences, which more candidates concentrating in the right tail;</div>
<div style="color: rgb(0, 0, 0);">→ Natural Sciences and Maths appear to have been the toughest subjects, with most of the candidates having a smaller grade, a bigger left tail;</div>
<div style="color: rgb(0, 0, 0);">→ The Languages grade distribution looks a lot like a normal distribution;</div>
</div>

## Checking normality

In [ ]:
def test_normality_ks_shapiro(
    df: pd.DataFrame,
    columns: List[str],
    alpha: float = 0.05
) -> pd.DataFrame:
    """
    Runs Kolmogorov–Smirnov and Shapiro–Wilk normality tests
    for each column and returns a consolidated table.

    Parameters
    ----------
    df : pd.DataFrame
        Dataset with numeric columns.
    columns : List[str]
        Columns to test.
    alpha : float
        Significance level.

    Returns
    -------
    pd.DataFrame
        Table summarizing normality test results.
    """

    results = []

    for col in columns:
        data = df[col].dropna()

        # Fit mean and std for KS
        mean, std = data.mean(), data.std()

        # Kolmogorov–Smirnov test
        ks_stat, ks_p = kstest(data, "norm", args=(mean, std))

        # Shapiro–Wilk (limit 5000 samples)
        if len(data) > 5000:
            data_sample = data.sample(5000, random_state=42)
        else:
            data_sample = data

        sh_stat, sh_p = shapiro(data_sample)

        results.append({
            "column": col,
            "ks_stat": ks_stat,
            "ks_p": ks_p,
            "ks_normal": ks_p > alpha,
            "shapiro_stat": sh_stat,
            "shapiro_p": sh_p,
            "shapiro_normal": sh_p > alpha,
            "normal_consensus": (ks_p > alpha) and (sh_p > alpha)
        })

    results_df = pd.DataFrame(results)
    print("Consolidated Normality Test Results (KS + Shapiro)")
    print(results_df)

    return results_df

In [ ]:
def plot_normality_distribution(
    df: pd.DataFrame,
    columns: List[str],
    bins: int = 40
) -> Figure:
    """
    Plots histogram + KDE + fitted normal curve for each column.
    """

    n = len(columns)
    fig, axes = plt.subplots(n, 1, figsize=(12, 4 * n))

    # Handle case where n == 1
    if n == 1:
        axes = [axes]

    for ax, col in zip(axes, columns):
        data = df[col].dropna()
        mean, std = data.mean(), data.std()

        # Histogram + KDE
        sns.histplot(
            data, bins=bins, kde=True, stat="density",
            color="#4A6FA5", alpha=0.6, ax=ax
        )

        # Fitted Normal Curve
        x = np.linspace(data.min(), data.max(), 300)
        y = norm.pdf(x, mean, std)
        ax.plot(x, y, color="red", linestyle="--", linewidth=2,
                label=f"Normal Fit (mean={mean:.2f}, stddev={std:.2f})")

        ax.set_title(f"Distribution of {col}", fontsize=15)
        ax.set_xlabel(col)
        ax.set_ylabel("Density")
        ax.legend()

    plt.tight_layout()
    plt.show()

In [ ]:
def plot_qq(
    df: pd.DataFrame,
    columns: List[str]
) -> Figure:
    """
    Creates QQ-plots for each selected column.
    """

    n = len(columns)
    fig, axes = plt.subplots(n, 1, figsize=(10, 4*n))

    if n == 1:
        axes = [axes]

    for ax, col in zip(axes, columns):
        data = df[col].dropna()

        probplot(data, dist="norm", plot=ax)
        ax.set_title(f"QQ-Plot for {col}", fontsize=15)
        ax.grid(True, alpha=0.3)

    plt.tight_layout()
    plt.show()

In [ ]:
results = test_normality_ks_shapiro(df, ['NU_NOTA_CN','NU_NOTA_CH','NU_NOTA_LC','NU_NOTA_MT'])

In [ ]:
plot_normality_distribution(df, ['NU_NOTA_CN','NU_NOTA_CH','NU_NOTA_LC','NU_NOTA_MT'])

In [ ]:
plot_qq(df, ['NU_NOTA_CN','NU_NOTA_CH','NU_NOTA_LC','NU_NOTA_MT'])

In [ ]:
# Creating columns for average grade
df_EDA["MEAN_GRADES"] = df_EDA[['NU_NOTA_CN', 'NU_NOTA_CH','NU_NOTA_LC','NU_NOTA_MT','NU_NOTA_REDACAO']].mean(axis=1)
df_private = df_EDA[df_EDA["TP_ESCOLA"]=="Private"]
df_public = df_EDA[df_EDA["TP_ESCOLA"]=="Public"]

# Plotting histogram
plt.figure(figsize=(15, 6))
ax = sns.histplot(data=df_public,x='MEAN_GRADES', bins=int(180/5), color='blue', edgecolor='black', label='Public')
ax = sns.histplot(data=df_private,x='MEAN_GRADES', bins=int(180/5), color='lightblue', edgecolor='black',label='Private')
ax.set(xlabel='Grades', ylabel='Number of candidates', title='Distribution of mean grades for private and public schools')
ax.legend()
# plt.grid(False)
plt.show()

In [ ]:
# Create a simplified column for school type.
df["type_school"] = df["TP_ESCOLA"].map({
    1: "Public",
    2: "Private",
    3: "Foreign"
})

# If there are missing values, fill in as "Public"
df["type_school"] = df["type_school"].fillna("Public")

In [ ]:
def categorize_income(v:int):
    '''
    Return the group according to the income bracket.
    '''
    # Q006 is family income in the ENEM exam (ascending order).
    if v == 3:
        return "Low"
    elif v == 6:
        return "Medium"
    else:
        return "High"

df["group_income"] = df["Q006"].astype('category').cat.codes.apply(categorize_income)

In [ ]:
def study_env(row):
    good_conditions = 0

    # Internet (Q025)
    if row["Q025"] in ["B"]:
        good_conditions += 1

    # Number of residents (Q005)
    if row["Q005"] <= 4:
        good_conditions += 1

    # Bathroom (Q006)
    if row["Q011"] in ["D", "E"]:
        good_conditions += 1

    return "Adequate" if good_conditions >= 2 else "Precarious"

df["study_env"] = df.apply(study_env, axis=1)

In [ ]:
from matplotlib.patches import Rectangle

PALETTE = {
    "Precarious": "#A7ADB7",
    "Adequate": "#3C4AFF"
}


df["school_income_group"] = (
    df["type_school"] + " / " + df["group_income"]
)

grupos_desejados = ["Public / Low", "Private / High"]
df_plot = df[df["school_income_group"].isin(grupos_desejados)].copy()

resumo = (
    df_plot
    .groupby(["school_income_group", "study_env"])["NU_NOTA_MT"]
    .mean()
    .reset_index()
)

resumo["school_income_group"] = pd.Categorical(
    resumo["school_income_group"],
    categories=["Public / Low", "Private / High"],
    ordered=True
)
resumo["study_env"] = pd.Categorical(
    resumo["study_env"],
    categories=["Precarious", "Adequate"],
    ordered=True
)


fig, ax = plt.subplots(figsize=(20,6))

bars = sns.barplot(
    data=resumo,
    y="school_income_group",
    x="NU_NOTA_MT",
    hue="study_env",
    palette=PALETTE,
    edgecolor="black",
    linewidth=1.2,
    ax=ax,
    orient="h"
)

# ---- Anotações elegantes nas barras ----
for p in bars.patches:
    width = p.get_width()
    ax.annotate(
        f"{width:.1f}",
        (width, p.get_y() + p.get_height() / 2),
        ha="left",
        va="center",
        fontsize=13,
        fontweight="bold",
        color="#252525",
        xytext=(8, 0),
        textcoords="offset points"
    )


ax.add_patch(Rectangle(
    (0, -0.5),
    max(resumo["NU_NOTA_MT"]) * 1.05,
    3,
    color="#F5F6F8",
    alpha=0.35,
    zorder=-1
))


ax.set_title(
    "Effect of Home Study Environment on Math Grades",
    fontsize=20, weight="bold", pad=18
)

ax.set_xlabel("Average Math Grade (NU_NOTA_MT)", fontsize=18)
ax.set_ylabel("High School Type / Socioeconomic Level", fontsize=18)

plt.xlim(0, max(resumo["NU_NOTA_MT"])*1.15)

plt.legend(
    title="Study Environment",
    title_fontsize=13,
    fontsize=13,
    frameon=False,
    loc="upper right"
)

plt.tight_layout()
plt.show()

In [ ]:
# Q006 is already an ordinal number (A, B, C...), let's map it to an order.
ordem_q006 = sorted(df["Q006"].dropna().unique())
map_q006 = {v: i for i, v in enumerate(ordem_q006)}
df["income_ord"] = df["Q006"].map(map_q006)

# Average grade by income level (order of Q006)
resumo_renda_mt = (
    df.groupby("income_ord")["NU_NOTA_MT"]
      .agg(["mean", "count"])
      .reset_index()
)

plt.figure(figsize=(11, 6))
ax = sns.lineplot(
    data=resumo_renda_mt,
    x="income_ord",
    y="mean",
    marker="o",
    linewidth=2.5
)
plt.title("Average Grade in Mathematics by Family Income Bracket (Q006)", fontsize=18, weight="bold")
plt.xlabel("Income Bracket (Q006 order)")
plt.ylabel("Average Grade in Mathematics")

# highlight points with text.
for _, row in resumo_renda_mt.iterrows():
    ax.annotate(
        f"{row['mean']:.0f}",
        (row["income_ord"], row["mean"]),
        textcoords="offset points",
        xytext=(0, 6),
        ha="center",
        fontsize=10
    )

plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(8,6))
sns.heatmap(df[["NU_NOTA_CN","NU_NOTA_CH","NU_NOTA_LC","NU_NOTA_MT","income_ord"]].corr(), annot=True)

In [ ]:
combined_df = pd.concat([df_public, df_private], ignore_index=True)
plt.figure(figsize=(15, 6))
ax = sns.countplot(data=combined_df,x='Q006',hue='TP_ESCOLA',palette=['blue',"lightblue"])
ax.set(xlabel='Family income classes', ylabel='Number of candidates', title='Amount of candidates from public/private school in each family income class')
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(15, 6))
classes = df_EDA["Q006"].unique().tolist()
mean_grades = df_EDA.groupby(["Q006"])['MEAN_GRADES'].mean().tolist()
plt.bar(classes, mean_grades)

ax.set_ylabel('Number of candidates')
ax.set_xlabel('Income class')
ax.set_title('Mean grade for all candidates from every family income class')

plt.show()

<div class="alert alert-info">
<strong><div style="color: rgb(0, 0, 0);">📌  Insights:</div></strong> <br>
<div style="color: rgb(0, 0, 0);">→ The distriution of grades of candidates from private schools are more to the left than for public schools;</div>
<div style="color: rgb(0, 0, 0);">→ The candidates that belong to lower family income classes also mostly go to public schools, while higer income classes go to private school. That is also common knowledge nowadays in Brazil, unfortunately, since the private school tend to offer a better education than public ones.;</div>
<div style="color: rgb(0, 0, 0);">→ The higher the family income class, the higher the candidates mean grades;</div>
</div>

# Data Preparation

Now that we already did the Exploratory Data Analysis, we should **drop features that may lead to a biased model**. The features indicating **sex** and **ethnicity** are typically the ones we drop in that case.

We know that someone's ethnicity shouldn't influence their grade in an exam per se. If we see a correlation, it shows characteristics of structural racism, in which ethnic minorities have less opportunities, in our case, that is reflected, for example, on the family income. So we will drop these two features. That way we can leave our model to focus on other features that provide more causation to the issue.

In [ ]:
df = df.drop(['TP_COR_RACA', 'TP_SEXO', 'SG_UF_ESC', 'TP_ST_CONCLUSAO','NU_NOTA_REDACAO', 'type_school', 'group_income', 'study_env', 'school_income_group', 'income_ord'],axis=1)

## Handling missing data

In [ ]:
df.dropna(inplace=True)

In [ ]:
print("Missing values in the dataset:",df.isnull().sum().sum())

## Handling *outliers*

In [ ]:
def remove_outliers_zscore(df, columns, threshold=3):
    """
    Removes outliers from specified columns using the Z-score method.

    Parameters:
    -----------
    df : pd.DataFrame
        Input dataset.
    columns : list
        List of numeric columns to apply Z-score filtering.
    threshold : float
        Z-score threshold to define an outlier (default = 3).

    Returns:
    --------
    pd.DataFrame
        DataFrame with outliers removed.
    """

    df_clean = df.copy()
    summary = []

    for col in columns:
        series = df_clean[col]

        # Compute z-scores
        mean = series.mean()
        std = series.std()

        z_scores = (series - mean) / std

        # Mask for values within threshold
        mask = z_scores.abs() <= threshold

        removed_count = (~mask).sum()
        removed_percent = round(100 * removed_count / len(series), 2)

        # Quartiles BEFORE removal
        Q1 = series.quantile(0.25)
        Q3 = series.quantile(0.75)

        summary.append({
            "column": col,
            "mean": mean,
            "std": std,
            "Q1": Q1,
            "Q3": Q3,
            "removed_rows": removed_count,
            "removed_percentage": removed_percent
        })

        # Apply filtering
        df_clean = df_clean[mask]

    # Print summary table
    summary_df = pd.DataFrame(summary)

    print("\n===== Z-Score Outlier Removal Summary =====")
    print(summary_df.to_string(index=False))
    print(f"\nTotal remaining rows: {len(df_clean)}")

    return df_clean

In [ ]:
def remove_outliers_IQR(df, columns, verbose=True):
    """
    Removes outliers from a DataFrame using the IQR rule.
    Works for one or multiple numeric columns.

    Parameters:
    -----------
    df : pd.DataFrame
        Input dataset.
    columns : list
        List of numeric columns from which outliers will be removed.
    verbose : bool
        If True, prints a summary of removed outliers.

    Returns:
    --------
    pd.DataFrame
        Cleaned DataFrame with outliers removed.
    pd.DataFrame
        Summary table of outliers removed per column.
    """

    df_clean = df.copy()
    summary = []

    for col in columns:
        series = df_clean[col].dropna()

        # Quartiles & IQR
        Q1 = series.quantile(0.25)
        Q3 = series.quantile(0.75)
        IQR = Q3 - Q1

        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR

        # Boolean mask of non-outliers
        mask = (df_clean[col] >= lower_bound) & (df_clean[col] <= upper_bound)

        removed = len(df_clean) - mask.sum()

        summary.append({
            "column": col,
            "Q1": Q1,
            "Q3": Q3,
            "IQR": IQR,
            "lower_bound": lower_bound,
            "upper_bound": upper_bound,
            "removed_rows": removed,
            "removed_percentage": round(100 * removed / len(df), 2)
        })

        # Keep only non-outliers
        df_clean = df_clean[mask]

    summary_df = pd.DataFrame(summary)

    if verbose:
        print("\n===== Outlier Removal Summary =====")
        print(summary_df.to_string(index=False))
        print(f"\nTotal removed rows: {len(df) - len(df_clean)}")
        print(f"Final dataset size: {len(df_clean)} rows")

    return df_clean

In [ ]:
df = remove_outliers_IQR(df, ['NU_NOTA_CN', 'NU_NOTA_CH', 'NU_NOTA_LC','NU_NOTA_MT'], verbose=True)
df.shape

## Correlation

In [ ]:
# Transforming categorical data to numeric
for i in range(1,28):
    if i < 10:
        name = f"Q00{i}"
    elif i == 26:
        continue
    else:
        name = f"Q0{i}"
    df[name] = df[name].astype('category').cat.codes

In [ ]:
plt.figure(figsize=(20,15))
sns.heatmap(df[['TP_FAIXA_ETARIA', 'TP_ESTADO_CIVIL', 'TP_ESCOLA', 'TP_ENSINO',
       'TP_LOCALIZACAO_ESC','NU_NOTA_CN', 'NU_NOTA_CH', 'NU_NOTA_LC',
       'NU_NOTA_MT','Q001', 'Q002', 'Q003', 'Q004', 'Q005', 'Q006',
       'Q007', 'Q008', 'Q009', 'Q010', 'Q011', 'Q012', 'Q013', 'Q014', 'Q015',
       'Q016', 'Q017', 'Q018', 'Q019', 'Q020', 'Q021', 'Q022', 'Q023', 'Q024',
       'Q025', 'Q027']].corr(), annot=False, cmap='coolwarm')

## Division into Training and Testing bases

In [ ]:
X = df.drop(['NU_NOTA_CN','NU_NOTA_CH','NU_NOTA_LC','NU_NOTA_MT'], axis=1)
y = df[['NU_NOTA_CN','NU_NOTA_CH','NU_NOTA_LC','NU_NOTA_MT']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [ ]:
print(f'X_train size: {X_train.shape}')
print(f'y_train size: {y_train.shape}')
print(f'X_test size: {X_test.shape}')
print(f'y_test size: {y_test.shape}')

# Modeling and Evaluation

In [ ]:
def evaluate_multioutput_regression(y_true, y_pred, target_names=None):
    """
    Evaluates a multi-output regression model using RMSE, MAE, and R²
    for each target column and returns a summary table.

    Parameters:
    -----------
    y_true : array-like
        True target values (n_samples, n_outputs)
    y_pred : array-like
        Predicted target values (n_samples, n_outputs)
    target_names : list or None
        List of target/column names (optional)

    Returns:
    --------
    pd.DataFrame
        Evaluation metrics per target and overall averages.
    """

    y_true = np.array(y_true)
    y_pred = np.array(y_pred)

    n_outputs = y_true.shape[1]

    if target_names is None:
        target_names = [f"target_{i}" for i in range(n_outputs)]

    metrics = []

    for i in range(n_outputs):
        rmse = np.sqrt(mean_squared_error(y_true[:, i], y_pred[:, i]))
        mae = mean_absolute_error(y_true[:, i], y_pred[:, i])
        r2 = r2_score(y_true[:, i], y_pred[:, i])

        metrics.append({
            "target": target_names[i],
            "RMSE": rmse,
            "MAE": mae,
            "R2": r2
        })

    df_results = pd.DataFrame(metrics)

    # Add averages
    avg_row = {
        "target": "AVERAGE",
        "RMSE": df_results["RMSE"].mean(),
        "MAE": df_results["MAE"].mean(),
        "R2": df_results["R2"].mean()
    }

    df_results = pd.concat([df_results, pd.DataFrame([avg_row])], ignore_index=True)

    print("\n===== Model Evaluation Summary =====")
    print(df_results.to_string(index=False))

    return df_results

In [ ]:
def plot_multioutput_diagnostics(y_true, y_pred, target_names):
    """
    Generates diagnostic plots for each target:
    1. Actual vs Predicted
    2. Residual distribution
    3. Residual boxplot
    4. Homoscedasticity check: residuals vs predicted + LOWESS smoothing
    """

    sns.set(style="whitegrid")

    y_true = np.array(y_true)
    y_pred = np.array(y_pred)

    # Garantir formato 2D
    if y_true.ndim == 1:
        y_true = y_true.reshape(-1, 1)
    if y_pred.ndim == 1:
        y_pred = y_pred.reshape(-1, 1)

    n_outputs = y_true.shape[1]

    fig, axes = plt.subplots(4, n_outputs, figsize=(6 * n_outputs, 22))

    # Se só tiver 1 coluna, axes vem 1D -> convertemos para 2D [4,1]
    if n_outputs == 1:
        axes = np.array(axes).reshape(4, 1)

    for i in range(n_outputs):
        true_vals = y_true[:, i]
        pred_vals = y_pred[:, i]
        residuals = true_vals - pred_vals

        # 1) Actual vs Predicted
        ax1 = axes[0, i]
        sns.scatterplot(x=true_vals, y=pred_vals, s=30, ax=ax1)
        ax1.plot(
            [true_vals.min(), true_vals.max()],
            [true_vals.min(), true_vals.max()],
            color="red",
            linestyle="--"
        )
        ax1.set_title(f"Actual vs Predicted – {target_names[i]}", fontsize=14, fontweight="bold")
        ax1.set_xlabel("Actual")
        ax1.set_ylabel("Predicted")

        # 2) Residual distribution
        ax2 = axes[1, i]
        sns.histplot(residuals, kde=True, ax=ax2)
        ax2.set_title(f"Residual Distribution – {target_names[i]}", fontsize=14, fontweight="bold")
        ax2.set_xlabel("Residuals")

        # 3) Residual boxplot
        ax3 = axes[2, i]
        sns.boxplot(x=residuals, ax=ax3)
        ax3.set_title(f"Residual Boxplot – {target_names[i]}", fontsize=14, fontweight="bold")
        ax3.set_xlabel("Residuals")

        # 4) Homoscedasticity Plot (Residuals vs Predicted)
        ax4 = axes[3, i]
        sns.scatterplot(x=pred_vals, y=residuals, s=25, alpha=0.5, ax=ax4)
        sns.regplot(x=pred_vals, y=residuals, scatter=False, ax=ax4, color="red")
        ax4.axhline(0, color="black", linestyle="--")
        ax4.set_title(f"Homoscedasticity Check – {target_names[i]}", fontsize=14, fontweight="bold")
        ax4.set_xlabel("Predicted")
        ax4.set_ylabel("Residuals")

    plt.tight_layout()
    plt.show()

In [ ]:
def get_feature_names(model, X_train):
    """
    Extracts feature names for models inside a Pipeline or raw estimators.
    Works with:
    - Pipeline(StandardScaler / OneHotEncoder / etc. + model)
    - Raw Scikit-learn models
    """
    if hasattr(model, "named_steps"):
        # If model is inside a Pipeline
        preprocessor = model.named_steps.get("scaler") or \
                       model.named_steps.get("preprocessor")

        if preprocessor and hasattr(preprocessor, "get_feature_names_out"):
            return preprocessor.get_feature_names_out()

    # Otherwise assume X_train is DataFrame
    if isinstance(X_train, pd.DataFrame):
        return X_train.columns.values

    return np.array([f"feature_{i}" for i in range(X_train.shape[1])])

In [ ]:
def shap_feature_importance(
    model,
    X,
    target_names=None,
    max_samples=3000,
    random_state=42,
    top_n=15,
    figsize=(10, 6),
    plot=True,
    plot_per_target=True
):
    """
    Computes SHAP feature importance for multi-output models, returning:

    - Global importance across outputs
    - Importance per output (target)
    - Plots for each individual target (optional)
    - Global plot (optional)
    """

    # Ensure DataFrame for feature names
    if not isinstance(X, pd.DataFrame):
        X = pd.DataFrame(X)

    # Sampling to reduce SHAP computation cost
    if len(X) > max_samples:
        X_sample = X.sample(max_samples, random_state=random_state)
    else:
        X_sample = X.copy()

    feature_names = X_sample.columns.tolist()

    # Detect multioutput
    is_multioutput = hasattr(model, "estimators_")

    if is_multioutput:
        n_targets = len(model.estimators_)
    else:
        y_sample = np.asarray(model.predict(X_sample.iloc[:5]))
        n_targets = 1 if y_sample.ndim == 1 else y_sample.shape[1]

    # Define target names
    if target_names is None:
        target_names = [f"target_{i}" for i in range(n_targets)]
    else:
        assert len(target_names) == n_targets, "Mismatch in number of target names."

    per_target_importance = {}
    overall_accumulator = np.zeros(len(feature_names))

    
    # shap for each output
    for i in range(n_targets):

        if is_multioutput:
            # One estimator per output
            est = model.estimators_[i]
            explainer = shap.Explainer(est, X_sample)
            shap_values = explainer(X_sample)
        else:
            # Single model with vector output → wrapper
            def predict_i(Xarr, estimator=model, idx=i):
                ypred = estimator.predict(Xarr)
                ypred = np.asarray(ypred)
                if ypred.ndim == 1:
                    return ypred
                return ypred[:, idx]

            explainer = shap.Explainer(predict_i, X_sample)
            shap_values = explainer(X_sample)

        values = shap_values.values  # shape = (n_samples, n_features)
        mean_abs = np.abs(values).mean(axis=0)  # 1D vector

        overall_accumulator += mean_abs

        # Per-output DataFrame
        df_imp = (
            pd.DataFrame({
                "feature": feature_names,
                "mean_abs_shap": mean_abs
            })
            .sort_values("mean_abs_shap", ascending=False)
            .reset_index(drop=True)
        )

        per_target_importance[target_names[i]] = df_imp

        # plot per output
        if plot and plot_per_target:

            top_df = df_imp.head(top_n).iloc[::-1]  # reverse for barh order

            plt.figure(figsize=figsize)
            plt.barh(top_df["feature"], top_df["mean_abs_shap"])
            plt.xlabel("Mean |SHAP value|")
            plt.ylabel("Feature")
            plt.title(f"SHAP Feature Importance — {target_names[i]}")
            plt.tight_layout()
            plt.show()

    
    # global importance - mean across output
    overall_mean = overall_accumulator / n_targets

    overall_df = (
        pd.DataFrame({
            "feature": feature_names,
            "mean_abs_shap": overall_mean
        })
        .sort_values("mean_abs_shap", ascending=False)
        .reset_index(drop=True)
    )

    # global plot
    if plot:
        top = overall_df.head(top_n).iloc[::-1]

        plt.figure(figsize=figsize)
        plt.barh(top["feature"], top["mean_abs_shap"])
        plt.xlabel("Mean |SHAP value| (average across outputs)")
        plt.ylabel("Feature")
        plt.title("Global SHAP Feature Importance (Across All Outputs)")
        plt.tight_layout()
        plt.show()

    return overall_df, per_target_importance

## Lasso Regression

In [ ]:
%%time

# Base Lasso model
lasso_base = Lasso(random_state=42, max_iter=10000)
model = MultiOutputRegressor(lasso_base)

# Hyperparameter grid
param_grid = {
    "estimator__alpha": [0.00001, 0.0001, 0.001, 0.01, 0.1, 1, 10]
}

# Cross-validation setup
cv = KFold(n_splits=10, shuffle=True, random_state=42)

# Halving Grid Search
halving_search = HalvingGridSearchCV(
    estimator=model,
    param_grid=param_grid,
    factor=3,    # how aggressively to eliminate weak models
    scoring="neg_mean_squared_error",
    cv=cv,
    n_jobs=-1,
    verbose=1
)

# Fit
halving_search.fit(X_train, y_train)

# Best params
print("Best parameters:", halving_search.best_params_)
print("Best score (neg MSE):", halving_search.best_score_)

best_rmse = np.sqrt(-halving_search.best_score_)
print("Best CV RMSE:", best_rmse)

# Best model predictions
best_lasso = halving_search.best_estimator_
y_pred = best_lasso.predict(X_test)

print('Lasso Regression training completed.')

In [ ]:
evaluate_multioutput_regression(y_test, y_pred, target_names=y_test.columns.to_list())

In [ ]:
plot_multioutput_diagnostics(y_test, y_pred, target_names=y_test.columns.to_list())

In [ ]:
shap_feature_importance(best_model, X_test, y_test.columns.to_list())

## Random Forest

In [ ]:
%%time

# Base model
rf_base = RandomForestRegressor(
    n_jobs=-1,
    random_state=42
)

rf_model = MultiOutputRegressor(rf_base)

# Hyperparameter grid
param_grid = {
    "estimator__n_estimators": [200, 400, 800],
    "estimator__max_depth": [5, 10, 15],
    "estimator__min_samples_split": [2, 5]
}

# Cross-validation
cv = KFold(n_splits=10, shuffle=True, random_state=42)

# Halving Grid Search
rf_search = HalvingGridSearchCV(
    estimator=rf_model,
    param_grid=param_grid,
    scoring="neg_mean_squared_error",
    factor=3,
    cv=cv,
    n_jobs=-1,
    verbose=1
)

rf_search.fit(X_train, y_train)

print("Best RF params:", rf_search.best_params_)
print("Best RF score (neg MSE):", rf_search.best_score_)
print("Best RF RMSE:", np.sqrt(-rf_search.best_score_))

best_rf = rf_search.best_estimator_
y_pred_rf = best_rf.predict(X_test)

print("Random Forest training completed.")

In [ ]:
evaluate_multioutput_regression(y_test, y_pred, target_names=y_test.columns.to_list())

In [ ]:
plot_multioutput_diagnostics(y_test, y_pred, target_names=y_test.columns.to_list())

## XGBoost

In [ ]:
%%time

# Base model
xgb_base = XGBRegressor(
    objective="reg:squarederror",
    n_jobs=-1,
    random_state=42
)

xgb_model = MultiOutputRegressor(xgb_base)

# Hyperparameter grid
param_grid = {
    "estimator__n_estimators": [300, 600, 900],
    "estimator__max_depth": [4, 6, 8],
    "estimator__learning_rate": [0.01, 0.05, 0.1],
    "estimator__subsample": [0.7, 0.9, 1.0],
    "estimator__colsample_bytree": [0.7, 0.9, 1.0],
}

# Cross-validation setup
cv = KFold(n_splits=5, shuffle=True, random_state=42)

# Halving Grid Search
xgb_search = HalvingGridSearchCV(
    estimator=xgb_model,
    param_grid=param_grid,
    factor=3,
    scoring="neg_mean_squared_error",
    cv=cv,
    n_jobs=-1,
    verbose=1
)

xgb_search.fit(X_train, y_train)

print("Best XGBoost params:", xgb_search.best_params_)
print("Best XGBoost score:", xgb_search.best_score_)
print("Best XGB RMSE:", np.sqrt(-xgb_search.best_score_))

best_xgb = xgb_search.best_estimator_
y_pred_xgb = best_xgb.predict(X_test)

print("XGBoost training completed.")

In [ ]:
evaluate_multioutput_regression(y_test, y_pred, target_names=y_test.columns.to_list())

In [ ]:
plot_multioutput_diagnostics(y_test, y_pred, target_names=y_test.columns.to_list())

In [ ]:
def evaluate_and_select_best_model(
    models: List[Tuple[str, Any]],
    X_train: pd.DataFrame,
    y_train: pd.DataFrame,
    X_test: pd.DataFrame,
    y_test: pd.DataFrame,
    pickle_filename: str = "best_model.pkl"
) -> Tuple[pd.DataFrame, Any]:
    """
    Evaluates multiple regression models using RMSE, MAE, and R2.
    Selects the best model (highest R2), saves it to a pickle file,
    and returns both the evaluation table and the best model object.

    Parameters
    ----------
    models : List[Tuple[str, Any]]
        A list of tuples: (model_name, model_instance).
    X_train : pd.DataFrame
        Training features.
    y_train : pd.DataFrame or pd.Series
        Training target(s).
    X_test : pd.DataFrame
        Test features.
    y_test : pd.DataFrame or pd.Series
        Test target(s).
    pickle_filename : str, optional
        Filename to save the best model as a pickle, by default "best_model.pkl".

    Returns
    -------
    Tuple[pd.DataFrame, Any]
        A tuple containing:
        - A DataFrame with RMSE, MAE, and R2 for each model
        - The best-performing model (highest R2)
    """

    results = []

    for name, model in models:
        # Train
        model.fit(X_train, y_train)

        # Predict
        y_pred = model.predict(X_test)

        # Metrics
        rmse = np.sqrt(mean_squared_error(y_test, y_pred))
        mae = mean_absolute_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)

        results.append({
            "model": name,
            "RMSE": rmse,
            "MAE": mae,
            "R2": r2
        })

    # Convert to DataFrame
    results_df = pd.DataFrame(results)
    print("\nModel Evaluation Results:\n")
    print(results_df)

    # Select the best model based on highest R2
    best_row = results_df.loc[results_df["R2"].idxmax()]
    best_model_name = best_row["model"]
    best_model = dict(models)[best_model_name]

    print(f"\nBest model: {best_model_name} (R2 = {best_row['R2']:.4f})")

    # Save best model
    with open(pickle_filename, "wb") as f:
        pickle.dump(best_model, f)

    print(f"\nBest model saved to: {pickle_filename}")

    return results_df, best_model


In [ ]:
models = [
    ("Lasso", best_lasso),
    ("Ridge", best_rf),
    ("RandomForest", best_xgb)
]

In [ ]:
results_df, best_model = evaluate_and_select_best_model(
    models,
    X_train, y_train,
    X_test, y_test,
    pickle_filename="best_regression_baseline_model.pkl"
)

## GNN Optimized

In [ ]:
SOCIO_COLS: List[str] = [
    "Q001","Q002","Q003","Q004","Q005","Q006",
    "Q007","Q008","Q009","Q010","Q011","Q012",
    "Q013","Q014","Q015","Q016","Q017","Q018",
    "Q019","Q020","Q021","Q022","Q023","Q024",
    "Q025","Q027"
]

TARGET_COLS = ["NU_NOTA_CN","NU_NOTA_CH","NU_NOTA_LC","NU_NOTA_MT"]

In [ ]:
def prepare_dataframe(df: pd.DataFrame) -> Tuple[pd.DataFrame, np.ndarray, StandardScaler]:
    df = df.reset_index(drop=True)

    # Targets
    y = df[TARGET_COLS].values.astype("float32")

    # Scale targets
    scaler_y = StandardScaler()
    y_scaled = scaler_y.fit_transform(y)

    return df, y_scaled, scaler_y


In [ ]:
def build_heterodata(df: pd.DataFrame) -> HeteroData:
    data = HeteroData()

    num_students = len(df)
    data["student"].num_nodes = num_students

    # Category nodes and edges
    for col in SOCIO_COLS:
        categories = df[col].astype(str).unique()
        mapping = {cat: i for i, cat in enumerate(categories)}

        df[f"{col}_id"] = df[col].astype(str).map(mapping)
        data[col].num_nodes = len(categories)

        src = df.index.values
        dst = df[f"{col}_id"].values

        data["student", f"has_{col}", col].edge_index = torch.tensor(
            [src, dst], dtype=torch.long
        )

        data[col, f"rev_has_{col}", "student"].edge_index = torch.tensor(
            [dst, src], dtype=torch.long
        )

    return data

In [ ]:
def add_knn_edges(df: pd.DataFrame, data: HeteroData, k: int = 5) -> HeteroData:
    socio_encoded = df[SOCIO_COLS].apply(
        lambda col: col.astype("category").cat.codes).values

    sim = cosine_similarity(socio_encoded)
    edges = []

    num_students = len(df)
    for i in range(num_students):
        nn_ids = np.argsort(sim[i])[-(k+1):-1]
        for j in nn_ids:
            edges.append([i, j])

    arr = np.array(edges).T
    data["student", "knn", "student"].edge_index = torch.tensor(arr, dtype=torch.long)
    data["student", "knn_rev", "student"].edge_index = torch.tensor(arr[[1, 0], :], dtype=torch.long)

    return data


In [ ]:
def create_splits(n: int):
    idx = np.arange(n)
    idx_train, idx_temp = train_test_split(idx, test_size=0.3, random_state=42)
    idx_val, idx_test = train_test_split(idx_temp, test_size=0.5, random_state=42)
    
    return (
        torch.tensor(idx_train, dtype=torch.long),
        torch.tensor(idx_val, dtype=torch.long),
        torch.tensor(idx_test, dtype=torch.long)
    )


In [ ]:
def create_loaders(data: HeteroData, idx_train, idx_val, idx_test, device):

    train_loader = NeighborLoader(
        data,
        input_nodes=("student", idx_train.to(device)),
        num_neighbors=[10, 10, 10],
        batch_size=512,
        shuffle=True
    )

    val_loader = NeighborLoader(
        data,
        input_nodes=("student", idx_val.to(device)),
        num_neighbors=[10, 10, 10],
        batch_size=512,
        shuffle=False
    )

    test_loader = NeighborLoader(
        data,
        input_nodes=("student", idx_test.to(device)),
        num_neighbors=[10, 10, 10],
        batch_size=512,
        shuffle=False
    )
    
    return train_loader, val_loader, test_loader


In [ ]:
class ENEMHeteroGNNOptimized(nn.Module):
    def __init__(self, data: HeteroData, emb_dim=64, hidden_dim=128, out_dim=4, dropout=0.4):
        super().__init__()

        self.embeddings = nn.ModuleDict({
            nt: nn.Embedding(data[nt].num_nodes, emb_dim)
            for nt in data.node_types
        })

        self.convs = nn.ModuleList()
        in_dim = emb_dim

        for _ in range(3):
            conv_dict = {et: SAGEConv((in_dim, in_dim), hidden_dim) for et in data.edge_types}
            self.convs.append(HeteroConv(conv_dict, aggr="sum"))
            in_dim = hidden_dim

        self.norm_student = nn.LayerNorm(hidden_dim)
        self.dropout = nn.Dropout(dropout)

        self.head = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim, out_dim)
        )

    def forward(self, data: HeteroData):
        device = next(self.parameters()).device
        x_dict = {}

        for nt in self.embeddings:
            if nt in data.node_types and hasattr(data[nt], "n_id"):
                node_ids = data[nt].n_id.to(device)
            else:
                node_ids = torch.arange(
                    self.embeddings[nt].num_embeddings,
                    device=device
                )
            x_dict[nt] = self.embeddings[nt](node_ids)

        for conv in self.convs:
            x_dict = conv(x_dict, data.edge_index_dict)
            x_dict = {nt: self.dropout(F.relu(v)) for nt, v in x_dict.items()}

        s = x_dict["student"]
        s = self.norm_student(s)
        out = self.head(s)

        return {"student": out}


In [ ]:
def save_checkpoint(path, model, optimizer, epoch, best_metric, scaler_y):
    torch.save({
        "model": model.state_dict(),
        "optimizer": optimizer.state_dict(),
        "epoch": epoch,
        "best_metric": best_metric,
        "scaler_y": scaler_y
    }, path)
    print(f"Checkpoint saved in {path}")


def load_checkpoint(path, model, optimizer, device="cpu"):
    ckpt = torch.load(path, map_location=device)
    model.load_state_dict(ckpt["model"])
    optimizer.load_state_dict(ckpt["optimizer"])
    print(f"Checkpoint loaded (epoch {ckpt['epoch']}, best R²={ckpt['best_metric']})")
    return ckpt


In [ ]:
def train_with_resume(
    model,
    optimizer,
    train_loader,
    val_loader,
    criterion,
    scaler_y,
    checkpoint_path,
    max_epochs=120,
    patience=20,
    device="cpu"
):
    if os.path.exists(checkpoint_path):
        ckpt = load_checkpoint(checkpoint_path, model, optimizer, device)
        start_epoch = ckpt["epoch"] + 1
        best_metric = ckpt["best_metric"]
        scaler_y = ckpt["scaler_y"]
        print(f"Starting from epocch {start_epoch}")
    else:
        start_epoch = 1
        best_metric = -np.inf
        print("Training from zero")

    wait = 0

    for epoch in range(start_epoch, max_epochs + 1):
        # Train
        model.train()
        losses = []

        for batch in train_loader:
            batch = batch.to(device)
            optimizer.zero_grad()

            out = model(batch)["student"]
            idx_global = batch["student"].n_id
            y_batch = y_scaled_tensor[idx_global]

            loss = criterion(out, y_batch)
            loss.backward()
            optimizer.step()

            losses.append(loss.item())

        train_loss = np.mean(losses)

        # Validation
        model.eval()
        preds, trues = [], []

        with torch.no_grad():
            for batch in val_loader:
                batch = batch.to(device)
                out = model(batch)["student"]
                idx_global = batch["student"].n_id
                y_batch = y_scaled_tensor[idx_global]

                preds.append(out.cpu().numpy())
                trues.append(y_batch.cpu().numpy())

        preds = scaler_y.inverse_transform(np.vstack(preds))
        trues = scaler_y.inverse_transform(np.vstack(trues))

        val_r2 = r2_score(trues, preds, multioutput="uniform_average")

        # Check improvement
        if val_r2 > best_metric:
            best_metric = val_r2
            wait = 0
            save_checkpoint(checkpoint_path, model, optimizer, epoch, best_metric, scaler_y)
            flag = "\nBEST"
        else:
            wait += 1
            flag = "   "

        print(f"{flag} Epoch {epoch} | Train Loss={train_loss:.4f} | Val R²={val_r2:.4f}")

        if wait >= patience:
            print("\nEarly stopping")
            break

    return best_metric


In [ ]:
def evaluate(model, loader, scaler_y, device="cpu"):
    model.eval()
    preds, trues = [], []

    with torch.no_grad():
        for batch in loader:
            batch = batch.to(device)
            out = model(batch)["student"]
            idx_global = batch["student"].n_id
            y_batch = y_scaled_tensor[idx_global]

            preds.append(out.cpu().numpy())
            trues.append(y_batch.cpu().numpy())

    preds = scaler_y.inverse_transform(np.vstack(preds))
    trues = scaler_y.inverse_transform(np.vstack(trues))

    return {
        "RMSE": mean_squared_error(trues, preds),
        "MAE": mean_absolute_error(trues, preds),
        "R2": r2_score(trues, preds)
    }


In [ ]:
def predict_student(model, df_row, hetero_data, scaler_y):
    model.eval()
    idx = df_row.name
    
    with torch.no_grad():
        full_out = model(hetero_data.to(next(model.parameters()).device))["student"]
        pred_scaled = full_out[idx].cpu().numpy().reshape(1, -1)
        pred_original = scaler_y.inverse_transform(pred_scaled)[0]

    return pred_original


In [ ]:
df, y_scaled, scaler_y = prepare_dataframe(df)

# Convert targets to global tensor
y_scaled_tensor = torch.tensor(y_scaled, dtype=torch.float32)


In [ ]:
hetero_data = build_heterodata(df)
hetero_data = add_knn_edges(df, hetero_data, k=5)

torch.save(hetero_data, "hetero_data.pt")
print("hetero_data.pt saved with success!")


In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
hetero_data = hetero_data.to(device)

In [ ]:
idx_train, idx_val, idx_test = create_splits(len(df))


In [ ]:
train_loader, val_loader, test_loader = create_loaders(
    hetero_data, idx_train, idx_val, idx_test, device
)


In [ ]:
model = ENEMHeteroGNNOptimized(hetero_data, emb_dim=64, hidden_dim=128, out_dim=4)
model = model.to(device)


In [ ]:
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
checkpoint_path = "best_model.pt"


In [ ]:
best_r2 = train_with_resume(
    model=model,
    optimizer=optimizer,
    train_loader=train_loader,
    val_loader=val_loader,
    criterion=criterion,
    scaler_y=scaler_y,
    checkpoint_path=checkpoint_path,
    max_epochs=1,
    patience=20,
    device=device
)

print("Melhor R²:", best_r2)

In [ ]:
metrics = evaluate(model, test_loader, scaler_y, device=device)
print(metrics)

In [ ]:
pred = predict_student(
    model=model,
    df_row=df.iloc[0],
    hetero_data=hetero_data,
    scaler_y=scaler_y
)

print("Prediction of the 4 tests:", pred)


# Deploy

In [ ]:
device = torch.device("cpu")

In [ ]:
class ENEMHeteroGNNOptimized(nn.Module):
    def __init__(self, data: HeteroData, emb_dim=64, hidden_dim=128, out_dim=4, dropout=0.4):
        super().__init__()

        self.embeddings = nn.ModuleDict({
            nt: nn.Embedding(data[nt].num_nodes, emb_dim)
            for nt in data.node_types
        })

        self.convs = nn.ModuleList()
        in_dim = emb_dim

        for _ in range(3):
            conv_dict = {et: SAGEConv((in_dim, in_dim), hidden_dim) for et in data.edge_types}
            self.convs.append(HeteroConv(conv_dict, aggr="sum"))
            in_dim = hidden_dim

        self.norm_student = nn.LayerNorm(hidden_dim)
        self.dropout = nn.Dropout(dropout)

        self.head = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim, out_dim)
        )

    def forward(self, data: HeteroData):
        device = next(self.parameters()).device
        x_dict = {}

        for nt in self.embeddings:
            if nt in data.node_types and hasattr(data[nt], "n_id"):
                node_ids = data[nt].n_id.to(device)
            else:
                node_ids = torch.arange(
                    self.embeddings[nt].num_embeddings,
                    device=device
                )
            x_dict[nt] = self.embeddings[nt](node_ids)

        for conv in self.convs:
            x_dict = conv(x_dict, data.edge_index_dict)
            x_dict = {nt: self.dropout(F.relu(v)) for nt, v in x_dict.items()}

        s = x_dict["student"]
        s = self.norm_student(s)
        out = self.head(s)

        return {"student": out}


In [ ]:
# load graph
hetero_data = torch.load("hetero_data.pt", map_location=device)
print("Graph loaded.")
print("Node types:", hetero_data.node_types)
print("Edge types:", hetero_data.edge_types)

# load checkpoint
ckpt = torch.load("best_model.pt", map_location=device)

# build model exaclty as the train
model = ENEMHeteroGNNOptimized(
    hetero_data,
    emb_dim=64,
    hidden_dim=128,
    out_dim=4,
    dropout=0.4
).to(device)

# load weights
model.load_state_dict(ckpt["model"])
model.eval()

# load scaler
scaler_y = ckpt["scaler_y"]

print("\nModel uploaded!")


In [ ]:
app = FastAPI(title="ENEM HeteroGNN Predictor")

class StudentRequest(BaseModel):
    student_id: int  # student index in the graph


In [ ]:
@app.post("/predict")
def predict(request: StudentRequest):

    student_id = request.student_id

    # verify limits
    if student_id >= hetero_data["student"].num_nodes:
        return {"error": "Student ID out of bounds."}

    # Forward pass
    with torch.no_grad():
        out_dict = model(hetero_data)
        preds = out_dict["student"][student_id].cpu().numpy()

    # desnormalize
    preds = scaler_y.inverse_transform(preds.reshape(1, -1))[0]

    return {
        "student_id": student_id,
        "predicted_scores": {
            "CN": float(preds[0]),
            "CH": float(preds[1]),
            "LC": float(preds[2]),
            "MT": float(preds[3])
        }
    }


In [ ]:
nest_asyncio.apply()

config = Config(
    app=app,
    host="0.0.0.0",
    port=8000,
    reload=False
)

server = Server(config)
await server.serve()